In [1]:
%matplotlib inline

''' original source for this tutorial
https://betatim.github.io/posts/sklearn-for-TMVA-users/
'''
print ("importing packages for python and ROOT")
import random

import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score

from root_numpy import root2array, rec2array

print ("importing packages is done")

importing packages for python and ROOT
Welcome to JupyROOT 6.12/06
importing packages is done


In [26]:
print ("importing branches now")

## List the branches names. 

#branch_names = ['PtL1', 'PtL2', 'PtL3', 'PtL4', 'PtJ1', 'PtJ2', 'MJ1', 'MJ2', 'nJet', 'nLep', 'MZ1', 'MZ2', 'PtZ1', 'PtZ2', 'MZZ', 'PtZZ', 'dPhiJJ', 'MJJ', 'EtaJ1xJ2', 'YZ1Star', 'YZ2Star', 'PtZZJJOHtZZJJ', 'Z1Cen', 'Z2Cen']
branch_names = ['CA15Puppi_massPruned','CA15Puppi_flavour','CA15Puppi_nbHadrons','CA15Puppi_ptPruned','CA15Puppi_etaPruned','CA15Puppi_SubJet_csv','CA15Puppi_z_ratio','CA15Puppi_trackSipdSig_3','CA15Puppi_trackSipdSig_2','CA15Puppi_trackSipdSig_1','CA15Puppi_trackSipdSig_0','CA15Puppi_trackSipdSig_1_0','CA15Puppi_trackSipdSig_0_0','CA15Puppi_trackSipdSig_1_1','CA15Puppi_trackSipdSig_0_1','CA15Puppi_trackSip2dSigAboveCharm_0','CA15Puppi_trackSip2dSigAboveBottom_0','CA15Puppi_trackSip2dSigAboveBottom_1','CA15Puppi_tau0_trackEtaRel_0','CA15Puppi_tau0_trackEtaRel_1','CA15Puppi_tau0_trackEtaRel_2','CA15Puppi_tau1_trackEtaRel_0','CA15Puppi_tau1_trackEtaRel_1','CA15Puppi_tau1_trackEtaRel_2','CA15Puppi_tau_vertexMass_0','CA15Puppi_tau_vertexEnergyRatio_0','CA15Puppi_tau_vertexDeltaR_0','CA15Puppi_tau_flightDistance2dSig_0','CA15Puppi_tau_vertexMass_1','CA15Puppi_tau_vertexEnergyRatio_1','CA15Puppi_tau_flightDistance2dSig_1','CA15Puppi_jetNTracks','CA15Puppi_nSV_']
"""lepton pT, lepton eta, lepton phi,
missing energy magnitude, missing energy phi,
jet 1 pt, jet 1 eta, jet 1 phi, jet 1 b-tag,
jet 2 pt, jet 2 eta, jet 2 phi, jet 2 b-tag,
jet 3 pt, jet 3 eta, jet 3 phi, jet 3 b-tag,
jet 4 pt, jet 4 eta, jet 4 phi, jet 4 b-tag,
m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb,
m_wwbb.split(",")
"""


branch_names = [c.strip() for c in branch_names]

branch_names = (b.replace(" ", "_") for b in branch_names)

branch_names = list(b.replace("-", "_") for b in branch_names)

#print (branch_names)

## convert ROOT Tree into a 2D array, sciKit works mainly on two d array. 
## 1 d is for variable name and second dimension for the values of those variables. 

## Open the rootfile and read TTree and convert those branches into a 2D record. And then convert this record into
## array. 
signal = root2array("ZpBaryonic_MZp-1000_MChi-1_trainNCUGlobalTuples_1.root",
                    "tree/treeMaker",
                    branch_names,
                    selection='CA15Puppi_ptPruned > 170.0 & ( CA15Puppi_massPruned < 400) & ( CA15Puppi_massPruned > 70)'
                   )

print ("signal")
print (signal)
signal = rec2array(signal)

backgr = root2array("QCD_500TO700_test.root",
                    "tree/treeMaker",
                    branch_names,
                    selection='CA15Puppi_ptPruned > 170.0 & ( CA15Puppi_massPruned < 400) & ( CA15Puppi_massPruned > 70)'
                   )
backgr = rec2array(backgr)


# for sklearn data is usually organised
# into one 2D array of shape (n_samples x n_features)
# containing all the data and one array of categories
# of length n_samples

# print ('number of branches ='), len(branch_names)
# print ( len(branch_names))
# print (signal.shape) 
# print (backgr.shape) 
## Join signal and background and everything is not contained in single entity. 
X = np.concatenate((signal, backgr))

print ("printing X shape"),(X.shape)
# print (X)
## add one more column, 1 is for signal and 0 is for background. 
y = np.concatenate((np.ones(signal.shape[0]),
                    np.zeros(backgr.shape[0])))
# print (y[0])
# print (signal.shape[0])
# print (np.ones(signal.shape[0]))
# print ((backgr.shape))
# print ("printing Y shape"),(y.shape)
# print (y.shape)


importing branches now
signal
[(array([118.7569], dtype=float32), array([-1.], dtype=float32), array([-1.], dtype=float32), array([303.4361], dtype=float32), array([2.6524055], dtype=float32), array([0.82922137], dtype=float32), array([8.858039], dtype=float32), array([-1.7178483], dtype=float32), array([-1.5741225], dtype=float32), array([0.9732842], dtype=float32), array([4.52087], dtype=float32), array([4.52087], dtype=float32), array([-50.], dtype=float32), array([0.9732842], dtype=float32), array([-50.], dtype=float32), array([-1.2277625], dtype=float32), array([-1.6190751], dtype=float32), array([-19.], dtype=float32), array([3.4530842], dtype=float32), array([3.5667782], dtype=float32), array([4.188206], dtype=float32), array([-1.], dtype=float32), array([-1.], dtype=float32), array([-1.], dtype=float32), array([4.7115393], dtype=float32), array([1.4210616], dtype=float32), array([0.06250124], dtype=float32), array([4.6057205], dtype=float32), array([-1.], dtype=float32), array(

In [16]:
from sklearn.model_selection import train_test_split
print ('size before spliting')
print ('xsize')
print (X.shape)
print ('ysize')
print (y.shape)
X_train,X_test,y_train,y_test = train_test_split (X, y, test_size=0.33, random_state=42)

print ('size after the spliting')
print ('xtrain')
print (X_train.shape)
print ('xtest')
print (X_test.shape)
print ('ytrain')
print (y_train.shape)
print ('ytest')
print (y_test.shape)

size before spliting
xsize
(75855, 33)
ysize
(75855,)
size after the spliting
xtrain
(50822, 33)
xtest
(25033, 33)
ytrain
(50822,)
ytest
(25033,)


In [19]:
print ("start training")
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, roc_auc_score

dt = DecisionTreeClassifier(max_depth=5)
bdt = AdaBoostClassifier(dt,algorithm='SAMME',n_estimators=800,learning_rate=0.5)
bdt.fit(X_train, y_train)
print ("bdt had done the fitting")

start training


ValueError: setting an array element with a sequence.

In [25]:
y_predicted = bdt.predict(X_test1)
print ("ypredicted")
print (y_predicted)

NotFittedError: This AdaBoostClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [26]:
print (classification_report(y_test1, y_predicted,
                            target_names=["background", "signal"]))

NameError: name 'y_predicted' is not defined

In [27]:
from sklearn.metrics import roc_curve, auc
decisions = bdt.decision_function(X_test1)
print (decisions)
print ("Area under the ROC: %.4f" %roc_auc_score(y_test1,decisions))

NotFittedError: This AdaBoostClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test1, decisions)
roc_auc = auc(fpr, tpr)

plt.plot(fpr, tpr,color='darkorange', lw=3, label='ROC (area = %0.2f)'%(roc_auc))

plt.plot([0, 1], [0, 1], '--', color='navy')
plt.xlim([-0.05, 1.05])
plt.ylim([-0.05, 1.05])
plt.xlabel(r'False Positive Rate')
plt.ylabel(r'True Positive Rate')
plt.title(r'Receiver operating characteristic')
plt.legend(loc="lower right")
plt.grid()
plt.show()
plt.savefig('roc')